<a href="https://colab.research.google.com/github/bigboykA/TVShow-Dropoff-Predictor/blob/main/Netflix_TV_show_dropoffPredictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("NetflixDatafinal.csv")
df.head()


df_clean = df.drop(columns=[
    'Timestamp',
    'What is your favorite tv show ',
    'What is your least favorite tv show',
    'If you stopped watching, how many episodes did you complete before on average.',
    'What was the main reason you stopped watching(if applicable)?'
])
df_clean.columns


df_clean.columns = [
    'age_range',
    'platform',
    'genre',
    'episode_length',
    'time_of_day',
    'device',
    'attention_start',
    'attention_end',
    'finished_show'
]

df_clean.head()



df_clean['finished_show'] = df_clean['finished_show'].map({'Yes': 1, 'No': 0})


df_clean['finished_show']

df_encoded = pd.get_dummies(df_clean, columns=[
    'platform',
    'genre',
    'device',
    'time_of_day',
    'episode_length',
    'age_range'
])


df_encoded.head()




df_encoded = df_encoded.dropna(subset=['finished_show'])

from sklearn.model_selection import train_test_split
X = df_encoded.drop('finished_show', axis=1)
y = df_encoded['finished_show']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

model = LogisticRegression(max_iter=1000)

# Fill missing values in the dataset
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)


model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))



#Train a Decision Tree Model
from sklearn.tree import DecisionTreeClassifier


tree_model = DecisionTreeClassifier(random_state=42)
tree_model.fit(X_train, y_train)


y_tree_pred = tree_model.predict(X_test)


from sklearn.metrics import accuracy_score, classification_report
print("Decision Tree Accuracy:", accuracy_score(y_test, y_tree_pred))
print(classification_report(y_test, y_tree_pred))

import matplotlib.pyplot as plt
import numpy as np

# Get feature importances and sort them
importances = tree_model.feature_importances_
feature_names = X_train.columns
sorted_idx = np.argsort(importances)[::-1]

# Plot
plt.figure(figsize=(12, 6))
plt.bar(range(len(importances)), importances[sorted_idx])
plt.xticks(range(len(importances)), feature_names[sorted_idx], rotation=90)
plt.title("Feature Importance (Decision Tree)")
plt.tight_layout()
plt.show()

#Random Forest

from sklearn.ensemble import RandomForestClassifier


rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)


y_rf_pred = rf_model.predict(X_test)

print("Random Forest Accuracy:", accuracy_score(y_test, y_rf_pred))
print(classification_report(y_test, y_rf_pred))

importances_rf = rf_model.feature_importances_
sorted_idx_rf = np.argsort(importances_rf)[::-1]

plt.figure(figsize=(12, 6))
plt.bar(range(len(importances_rf)), importances_rf[sorted_idx_rf])
plt.xticks(range(len(importances_rf)), feature_names[sorted_idx_rf], rotation=90)
plt.title("Feature Importance (Random Forest)")
plt.tight_layout()
plt.show()



from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Create confusion matrix
cm = confusion_matrix(y_test, y_rf_pred)

# Plot confusion matrix
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["No", "Yes"], yticklabels=["No", "Yes"])
plt.title("Confusion Matrix (Random Forest)")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

from sklearn.metrics import roc_curve, auc

# Get probabilities for positive class (1 = finished)
y_rf_probs = rf_model.predict_proba(X_test)[:, 1]

# Compute ROC curve and AUC
fpr, tpr, thresholds = roc_curve(y_test, y_rf_probs)
roc_auc = auc(fpr, tpr)

# Plot ROC
plt.figure(figsize=(6, 5))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve (Random Forest)')
plt.legend(loc="lower right")
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'NetflixDatafinal.csv'